# Lotka-Volterra example: the permanent case

## Definition of the optimal control problem

$$
    \left\{
    \begin{array}{l}
        \displaystyle \min x_3(T), \\[0.5em]
        \dot{x}_1(t) = r x_1(t) - x_1(t) x_2(t), \\[0.5em]
        \dot{x}_2(t) = -m x_2(t) +x_1(t) x_2(t) - u(t) x_2(t),  \\[0.5em]
        \dot{x}_3(t) = 1_{K^c}(x_1(t),x_2(t)),  \\[0.5em]
        u(t) \in [0, u_{\mathrm{max}}],\\[0.5em]
        x(0) = (x_0,y_0,0).
    \end{array}
    \right.
$$

with $K$ denotes $\{(x,y) \mid x\geq s\}$ with $u_{\mathrm{max}}+m< s $.

In [23]:
# For direct methods
using JuMP  # NLP modeling
using Ipopt # NLP solving
# To plot solutions
using Plots
using Plots.PlotMeasures

In [24]:
mutable struct Directt
    t; x1; x2 ; λ ; u ; xu ; xv 
end

In [25]:
# Parameters
t0  = 0.    # initial time
tf  = 5.
M   = 1.0;  # control bound u_max
m   = 0.5;
r   = 1.1;
s  = 7.;

In [26]:
function F(x, alpha, a)
    return 1 / (1 + exp(alpha * (x - a)))
end
fNC(x)  = F(x, 170, r)
fC(x)   = 1. - F(x, 170, r)

plot(fNC, -0.5, 5, color="red",  label="fNC", size=(400, 300));

### Direct method

In [27]:
function LV(x0 , ϵ; solution=[], nsteps=50, display=true)

    # Create JuMP model, using Ipopt as the solver
    if display
        pl = 5
    else
        pl = 1
    end
    sys = Model(optimizer_with_attributes(Ipopt.Optimizer, "print_level" => pl))
    set_optimizer_attribute(sys,"tol",1e-8)
    set_optimizer_attribute(sys,"constr_viol_tol",1e-8)
    set_optimizer_attribute(sys,"max_iter",1000)

    N  = nsteps     # Grid size

    @variables(sys, begin
                x1[1:N+1]    
                x2[1:N+1]   
                x3[1:N+1]   
         0.0  ≤  λ[1:N+1]  ≤ M
         0.0  ≤  u[1:N+1]  ≤ M
         0.0  ≤ Δt         ≤ 1.0 
        -1e1  ≤   v[1:N+1] ≤ 1e1 
         -1e1  ≤ xv[1:N+1]
         -1e1  ≤ xu[1:N+1]
    end)
    
    set_start_value(Δt, 20/N);
    for i in 1:N+1
        set_start_value(λ[i] , 1.0);
        set_start_value(v[i] , 0.0);
        set_start_value(xv[i], 0.0);
        set_start_value(xu[i], 0.0);
    end
    
    T = Δt*(N+1) ;

    # Objective
    @objective(sys, Min, x3[N+1] + T + ε*xv[N+1] + xu[N+1])

    # Boundary constraints
    @constraints(sys, begin
        con_xu0, xu[1]   == 0.0
        con_xv0, xv[1]   == 0.0
        con_x10, x1[1]   == x0[1] 
        con_x20, x2[1]   == x0[2] 
        con_x30, 0. <= λ[1]    <= M
        con_x3f, 0. <= λ[N+1]  <= M
    end)
    
    # Dynamics
    @NLexpression(sys, dx1[j = 1:N+1], r*x1[j] - x1[j]*x2[j])
    @NLexpression(sys, dx2[j = 1:N+1], -m*x2[j] + x1[j]*x2[j] - fC(x1[j])*u[j]*x2[j] - - fNC(x1[j])*λ[j]*x2[j])
    @NLexpression(sys,  dλ[j = 1:N+1], fC(x1[j])*v[j])
    @NLexpression(sys, dxv[j = 1:N+1], v[j]^2)
    @NLexpression(sys, dxu[j = 1:N+1], fNC(x1[j])*u[j]^2)
    
    # Dynamics with Crank-Nicolson scheme
    @NLconstraints(sys, begin
        con_dx1[j=1:N], x1[j+1] == x1[j] + 0.5 * Δt * (dx1[j+1] + dx1[j])
        con_dx2[j=1:N], x2[j+1] == x2[j] + 0.5 * Δt * (dx2[j+1] + dx2[j])
        con_dx3[j=1:N],  λ[j+1] ==  λ[j] + 0.5 * Δt * (dλ[j+1]  + dλ[j])
        con_dxv[j=1:N], xv[j+1] == xv[j] + 0.5 * Δt * (dxv[j+1] + dxv[j])
        con_dxu[j=1:N], xu[j+1] == xu[j] + 0.5 * Δt * (dxu[j+1] + dxu[j])
    end)

 

    # Solve for the control and state
    if display
        println("Solving...")
    end
    status = optimize!(sys)
    if display
        println()
    end

    # Display results
    if display
        if termination_status(sys) == MOI.OPTIMAL
            println("  Solution is optimal")
        elseif  termination_status(sys) == MOI.LOCALLY_SOLVED
            println("  (Local) solution found")
        elseif termination_status(sys) == MOI.TIME_LIMIT && has_values(sys)
            println("  Solution is suboptimal due to a time limit, but a primal solution is available")
        else
            error("  The model was not solved correctly.")
        end
        println("  objective value = ", objective_value(sys))
        println()
    end

    # Retrieves values (including duals)
    x1 = value.(x1)[:]
    x2 = value.(x2)[:]
    x3 = value.(x3)[:]
    u  = value.(u)[:]
    t  = (0:N) * value.(Δt)

  
    return Directt(t , x1 , x2 , λ , u , xu , xv )
end;

In [28]:
# Resolution
x0  = [0.5; 0.5]

ε   = 1e-4

sol = LV(x0, ε, nsteps=500);

Solving...
This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:    16504
Number of nonzeros in inequality constraint Jacobian.:        4
Number of nonzeros in Lagrangian Hessian.............:    48000

Total number of variables............................:     4009
                     variables with only lower bounds:     1002
                variables with lower and upper bounds:     1504
                     variables with only upper bounds:        0
Total number of equality constraints.................:     2504
Total number of inequality constraints...............:        4
        inequality constraints with only lower bounds:        2
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        2

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  2.0040000e+01 5.00e-01 1.00e+02  -1.0 0.00e+00    -  0.00e+00

┌ Warning: Function fNC automatically registered with 1 arguments.
│ 
│ Calling the function with a different number of arguments will result in an
│ error.
│ 
│ While you can safely ignore this warning, we recommend that you manually
│ register the function as follows:
│ ```Julia
│ model = Model()
│ register(model, :fNC, 1, fNC; autodiff = true)
│ ```
└ @ MathOptInterface.Nonlinear ~/.julia/packages/MathOptInterface/fTxO0/src/Nonlinear/operators.jl:370
┌ Warning: Function fC automatically registered with 1 arguments.
│ 
│ Calling the function with a different number of arguments will result in an
│ error.
│ 
│ While you can safely ignore this warning, we recommend that you manually
│ register the function as follows:
│ ```Julia
│ model = Model()
│ register(model, :fC, 1, fC; autodiff = true)
│ ```
└ @ MathOptInterface.Nonlinear ~/.julia/packages/MathOptInterface/fTxO0/src/Nonlinear/operators.jl:370


LoadError:   The model was not solved correctly.

In [8]:
# Plots
x1 = direct_sol.x1
x2 = direct_sol.x2
u  = direct_sol.u


x1_plot   = plot(x1,  xlabel = "t", ylabel = "x1",  legend = false)
x2_plot   = plot(x2,  xlabel = "t", ylabel = "x2",  legend = false)
u_plot    = plot( u,   xlabel = "t", ylabel = "u",   legend = false, size=(800,400)) #, linetype=:steppre)
x1x2_plot = plot(x1, x2, xlabel = "x1", ylabel = "x2",  legend = false)

plot(x1_plot, x2_plot, x1x2_plot, u_plot, layout = (2,3), size=(1200,800), left_margin=5mm)

LoadError: UndefVarError: direct_sol not defined

### Indirect method

In [9]:
using NLsolve
include("flow.jl");

In [10]:
# Dynamics
function F0(x)
    return [ x[2], 0.0 ]
end

function F1(x)
    return [ 0., 1. ]
end

# Hamiltonians and associated flows
H0(x, p) = p' * F0(x)
H1(x, p) = p' * F1(x)

H(x, p, u) = H0(x, p) + u*H1(x,p) # pseudo-Hamiltonian

up(x, p) =  M
um(x, p) = -M

Hp(x, p) = H(x, p, up(x, p))
Hm(x, p) = H(x, p, um(x, p))


fp = Flow(Hp)
fm = Flow(Hm);

In [11]:
# Shooting function
function shootBpBm(p0, t1, T, x0) # B+ B- structure
    
    x1, p1 = fp(t0, x0, p0, t1)
    xT, pT = fm(t1, x1, p1, T)
    
    s = zeros(eltype(p0), 4)
    
    s[1:2] = xT - [ x1f, x2f ] # target
    
    s[3] = H1(x1, p1) # switching
    s[4] = Hp(x0, p0) - 1. # free final time

    return s

    end;

In [12]:
# Initialisation
m = length(t)
x = [ [  x1[i],  x2[i] ] for i in 1:m ]
p = [ [  p1[i],  p2[i] ] for i in 1:m ];
H1s = H1.(x, p)

H1_plot = plot(t, H1s, xlabel = "t", ylabel = "H1", legend = false)

display(plot(u_plot, H1_plot, layout = (1,2), size=(800,400)))

it1 = findall(H1s[2:m].*H1s[1:m-1].<0.)
if length(it1)==1
    it1 = it1[1]
else
    error("Should have only one switch")
end

p0 = p[1]
t1 = t[it1]
T  = t[end]
ξ  = [ p0 ; t1 ; T ]
println("Initial guess:\n", ξ)

LoadError: UndefVarError: t not defined

Some of the types have been truncated in the stacktrace for improved reading. To emit complete information
in the stack trace, evaluate `TruncatedStacktraces.VERBOSE[] = true` and re-run the code.


In [13]:
# Solve
S(ξ) = shootBpBm(ξ[1:2], ξ[3], ξ[4], x0_ref)

println("Initial value of shooting:\n", S(ξ), "\n\n")

indirect_sol = nlsolve(S, ξ; xtol=1e-8, method=:trust_region, show_trace=true)
println(indirect_sol)

# Retrieves solution
if indirect_sol.f_converged || indirect_sol.x_converged
    p0 = indirect_sol.zero[1:2]
    t1 = indirect_sol.zero[3]
    T  = indirect_sol.zero[4]
else
    error("Not converged")
end;

LoadError: UndefVarError: ξ not defined

Some of the types have been truncated in the stacktrace for improved reading. To emit complete information
in the stack trace, evaluate `TruncatedStacktraces.VERBOSE[] = true` and re-run the code.


In [14]:
# plots

ode_sol = fp((t0, t1), x0_ref, p0, saveat=0.1)

tt1 = ode_sol.t
xx1 = [ ode_sol[1:2, j] for j in 1:size(tt1, 1) ]
pp1 = [ ode_sol[3:4, j] for j in 1:size(tt1, 1) ]
uu1 = up.(xx1, pp1)

ode_sol = fm((t1, T), xx1[end], pp1[end], saveat=0.1)

tt2 = ode_sol.t
xx2 = [ ode_sol[1:2, j] for j in 1:size(tt2, 1) ]
pp2 = [ ode_sol[3:4, j] for j in 1:size(tt2, 1) ]
uu2 = um.(xx2, pp2)

t = [tt1; tt2]
x = [ xx1 ; xx2 ]
p = [ pp1 ; pp2 ]
u = [ uu1 ; uu2 ]

m = length(t)
x1 = [ x[i][1] for i=1:m ]
x2 = [ x[i][2] for i=1:m ]
p1 = [ p[i][1] for i=1:m ]
p2 = [ p[i][2] for i=1:m ];

x1_plot   = plot(t,  x1, xlabel = "t", ylabel = "x1",  legend = false)
x2_plot   = plot(t,  x2, xlabel = "t", ylabel = "x2",  legend = false)
p1_plot   = plot(t,  p1, xlabel = "t", ylabel = "p1", legend = false)
p2_plot   = plot(t,  p2, xlabel = "t", ylabel = "p2", legend = false)
u_plot    = plot(t,   u, xlabel = "t", ylabel = "u",   legend = false, size=(800,400)) #, linetype=:steppre)
x1x2_plot = plot(x1, x2, xlabel = "x1", ylabel = "x2",  legend = false)

plot(x1_plot, x2_plot, x1x2_plot, p1_plot, p2_plot, u_plot, layout = (2,3), size=(1200,800), left_margin=5mm)

LoadError: UndefVarError: t1 not defined

Some of the types have been truncated in the stacktrace for improved reading. To emit complete information
in the stack trace, evaluate `TruncatedStacktraces.VERBOSE[] = true` and re-run the code.
